In [1]:
#import libraries
import pandas as pd
import numpy as np
import re

In [2]:
#open manufacturing workers data
df = pd.read_csv("data/manufacturing-workers-mexico-total.csv", skiprows=4)

In [3]:
#show data
df

,Periodos,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Total nacional /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Ensenada /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Mexicali /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Tecate /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Baja California > Tijuana /f1 /a","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Coahuila de Zaragoza > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Coahuila de Zaragoza > Acu�a /f1",...,"Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Total /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Matamoros /f1","Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Tamaulipas > Nuevo Laredo /f1","Manufacturas > Industria manufacturera, maquiladora y de serv

In [4]:
#drop final rows
df = df.drop([174,175,176])

In [5]:
#untidy data
df_melted = pd.melt(df, id_vars=["Periodos"], var_name ="State, City", value_name ="Manufacturing workers").copy()
df_melted

,Periodos,"State, City",Manufacturing workers
0,2007/07,"Manufacturas > Industria manufacturera, maquil...",1912745.0
1,2007/08,"Manufacturas > Industria manufacturera, maquil...",1929146.0
2,2007/09,"Manufacturas > Industria manufacturera, maquil...",1940574.0
3,2007/10,"Manufacturas > Industria manufacturera, maquil...",1953906.0
4,2007/11,"Manufacturas > Industria manufacturera, maquil...",1936734.0
...,...,...,...
10435,2021/08,"Manufacturas > Industria manufacturera, maquil...",139953.0
10436,2021/09,"Manufacturas > Industria manufacturera, maquil...",138520.0
10437,2021/10,"Manufacturas > Industria manufacturera, maquil...",140077.0
10438,2021/11,"Manufacturas > Industria manufacturera, maquil...",143237.0


In [6]:
rows = df_melted.shape[0]
rows

10440

In [7]:
df_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n (IMMEX) > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1'

In [8]:
#clean data (parentesis)
for i in range(rows):
    df_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', df_melted["State, City"][i])

<ipython-input-8-c4ca17155f0d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted["State, City"][i]= re.sub(r'\([^)]*\)', '', df_melted["State, City"][i])


In [9]:
df_melted["State, City"][220]

'Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > Aguascalientes /f1'

In [10]:
#clean data (other text)
df_clean= df_melted.replace("Manufacturas > Industria manufacturera, maquiladora y de servicios de exportaci�n  > Por entidad federativa > Establecimientos manufactureros > Personal ocupado por calificaci�n de mano de obra y condici�n de contrataci�n seg�n entidades federativas y municipios > Total personal ocupado > ","",regex=True)
df_clean= df_clean.replace(" /f1","",regex=True)
df_clean= df_clean.replace(">",",",regex=True)
df_clean

,Periodos,"State, City",Manufacturing workers
0,2007/07,Total nacional,1912745.0
1,2007/08,Total nacional,1929146.0
2,2007/09,Total nacional,1940574.0
3,2007/10,Total nacional,1953906.0
4,2007/11,Total nacional,1936734.0
...,...,...,...
10435,2021/08,Otras entidades federativas,139953.0
10436,2021/09,Otras entidades federativas,138520.0
10437,2021/10,Otras entidades federativas,140077.0
10438,2021/11,Otras entidades federativas,143237.0


In [11]:
#split columns
df_clean["Month"] = df_clean["Periodos"].str.split('/', expand=True)[1]
df_clean["Year"] = df_clean["Periodos"].str.split('/', expand=True)[0]
df_clean["State"] = df_clean["State, City"].str.split(',', expand=True)[0]
df_clean["City"] = df_clean["State, City"].str.split(',', expand=True)[1]
df_clean.drop(["Periodos","State, City"], axis=1)

,Manufacturing workers,Month,Year,State,City
0,1912745.0,07,2007,Total nacional,None
1,1929146.0,08,2007,Total nacional,None
2,1940574.0,09,2007,Total nacional,None
3,1953906.0,10,2007,Total nacional,None
4,1936734.0,11,2007,Total nacional,None
...,...,...,...,...,...
10435,139953.0,08,2021,Otras entidades federativas,None
10436,138520.0,09,2021,Otras entidades federativas,None
10437,140077.0,10,2021,Otras entidades federativas,None
10438,143237.0,11,2021,Otras entidades federativas,None


In [12]:
#arrange columns
df_clean = df_clean[["Year","Month", "State","City", "Manufacturing workers"]]
df_clean

,Year,Month,State,City,Manufacturing workers
0,2007,07,Total nacional,None,1912745.0
1,2007,08,Total nacional,None,1929146.0
2,2007,09,Total nacional,None,1940574.0
3,2007,10,Total nacional,None,1953906.0
4,2007,11,Total nacional,None,1936734.0
...,...,...,...,...,...
10435,2021,08,Otras entidades federativas,None,139953.0
10436,2021,09,Otras entidades federativas,None,138520.0
10437,2021,10,Otras entidades federativas,None,140077.0
10438,2021,11,Otras entidades federativas,None,143237.0


In [13]:
df_clean["State"].nunique()

19

In [14]:
df_clean["State"].unique()

array(['Total nacional', 'Aguascalientes', 'Baja California ',
       'Coahuila de Zaragoza ', 'Chihuahua ', 'Ciudad de M�xico',
       'Durango', 'Guanajuato ', 'Jalisco ', 'M�xico ', 'Nuevo Le�n ',
       'Puebla ', 'Quer�taro ', 'San Luis Potos� ', 'Sonora ',
       'Tamaulipas ', 'Veracruz de Ignacio de la Llave', 'Yucat�n ',
       'Otras entidades federativas'], dtype=object)

In [15]:
#clean other characters
df_clean = df_clean.replace("M�xico","Mexico",regex=True)
df_clean = df_clean.replace("Le�n","Leon",regex=True)
df_clean = df_clean.replace("Quer�taro","Queretaro",regex=True)
df_clean = df_clean.replace("Potos�","Potosi",regex=True)
df_clean = df_clean.replace("Yucat�n","Yucatan",regex=True)


In [16]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers
9232,2008,05,Tamaulipas,Reynosa,97514.0
7054,2015,05,Queretaro,Total,80895.0
1380,2021,01,Coahuila de Zaragoza,Total,258267.0
9253,2010,02,Tamaulipas,Reynosa,75094.0
3690,2010,07,Guanajuato,Otros municipios,59066.0
2069,2020,06,Coahuila de Zaragoza,Torre�n,30824.0
1824,2014,07,Coahuila de Zaragoza,Saltillo,33784.0
183,2008,04,Aguascalientes,None,38448.0
8952,2014,01,Tamaulipas,Matamoros,47420.0
4616,2015,03,Mexico,Total,134081.0


In [17]:
df_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate', ' Tijuana /a',
       ' Acu�a', ' Ramos Arizpe', ' Saltillo', ' Torre�n',
       ' Otros municipios', ' Chihuahua', ' Ju�rez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicol�s de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' M�rida'], dtype=object)

In [18]:
#clean other characters
df_clean = df_clean.replace("Acu�a","Acuña",regex=True)
df_clean = df_clean.replace("Torre�n","Torreon",regex=True)
df_clean = df_clean.replace("Ju�rez","Juarez",regex=True)
df_clean = df_clean.replace("Nicol�s","Nicolas",regex=True)
df_clean = df_clean.replace("M�rida","Merida",regex=True)
df_clean = df_clean.replace(" /a","",regex=True)


In [19]:
df_clean["City"].unique()

array([None, ' Total', ' Ensenada', ' Mexicali', ' Tecate', ' Tijuana',
       ' Acuña', ' Ramos Arizpe', ' Saltillo', ' Torreon',
       ' Otros municipios', ' Chihuahua', ' Juarez', ' Leon',
       ' Guadalajara', ' Zapopan', ' Tlalnepantla de Baz', ' Toluca',
       ' Apodaca', ' Guadalupe', ' Monterrey',
       ' San Nicolas de los Garza', ' Santa Catarina', ' Puebla',
       ' Queretaro', ' San Luis Potosi', ' Hermosillo', ' Nogales',
       ' Matamoros', ' Nuevo Laredo', ' Reynosa', ' Merida'], dtype=object)

In [20]:
#remove none values
for z in range(rows):
    if df_clean["City"][z] == None:
        df_clean["City"][z] = "Total"

<ipython-input-20-cc734ebbda36>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["City"][z] = "Total"


In [21]:
#remove blank spaces at the end
for a in range(rows):
    df_clean["State"][a] = df_clean["State"][a].rstrip().lstrip()
    df_clean["City"][a] = df_clean["City"][a].rstrip().lstrip()

<ipython-input-21-185b7ceb8dc1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["State"][a] = df_clean["State"][a].rstrip().lstrip()
<ipython-input-21-185b7ceb8dc1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["City"][a] = df_clean["City"][a].rstrip().lstrip()


In [22]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers
7396,2014,11,Queretaro,Otros municipios,39830.0
669,2019,10,Baja California,Ensenada,21988.0
3648,2021,07,Guanajuato,Leon,22394.0
6254,2021,03,Nuevo Leon,Santa Catarina,26243.0
5451,2012,04,Nuevo Leon,Apodaca,67691.0
10411,2019,08,Otras entidades federativas,Total,145539.0
5735,2021,06,Nuevo Leon,Guadalupe,38851.0
3002,2011,03,Ciudad de Mexico,Total,49060.0
3774,2017,07,Guanajuato,Otros municipios,119283.0
752,2012,03,Baja California,Mexicali,48504.0


In [23]:
#flag border states and cities
border = ["Baja California", "Coahuila de Zaragoza", "Chihuahua", "Nuevo Leon", "Tamaulipas", "Sonora"]
f = len(border)
df_clean["Border"] = "NA"
for k in range(rows):
    p = 0
    for l in range(f):
        if df_clean["State"][k] == border[l]:
            p = p +1
    if p > 0:
        df_clean["Border"][k] = "Yes"
    else:
        df_clean["Border"][k] = "No"

<ipython-input-23-7c27046fdcff>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Border"][k] = "No"
<ipython-input-23-7c27046fdcff>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["Border"][k] = "Yes"


In [24]:
df_clean.sample(20)

,Year,Month,State,City,Manufacturing workers,Border
10172,2014,03,Yucatan,Otros municipios,12882.0,No
1725,2020,10,Coahuila de Zaragoza,Ramos Arizpe,56187.0,Yes
1250,2010,03,Coahuila de Zaragoza,Total,146782.0,Yes
1258,2010,11,Coahuila de Zaragoza,Total,165436.0,Yes
3040,2014,05,Ciudad de Mexico,Total,41629.0,No
4401,2011,10,Jalisco,Otros municipios,51587.0,No
6551,2016,12,Puebla,Total,76645.0,No
1167,2017,10,Baja California,Tijuana,230605.0,Yes
7888,2012,05,San Luis Potosi,Otros municipios,10442.0,No
1170,2018,01,Baja California,Tijuana,222069.0,Yes


In [25]:
df_clean.groupby("Border")["Border"].agg("count")

Border
No     5046
Yes    5394
Name: Border, dtype: int64

In [26]:
df_clean.query("State == 'Baja California'")

,Year,Month,State,City,Manufacturing workers,Border
348,2007,07,Baja California,Total,257798.0,Yes
349,2007,08,Baja California,Total,260827.0,Yes
350,2007,09,Baja California,Total,261260.0,Yes
351,2007,10,Baja California,Total,262512.0,Yes
352,2007,11,Baja California,Total,259918.0,Yes
...,...,...,...,...,...,...
1213,2021,08,Baja California,Tijuana,266351.0,Yes
1214,2021,09,Baja California,Tijuana,268704.0,Yes
1215,2021,10,Baja California,Tijuana,266005.0,Yes
1216,2021,11,Baja California,Tijuana,262993.0,Yes


In [27]:
#Export to CSV
df_clean.to_csv("data/manufacturing-workers-mexico-refined.csv", index = False, encoding = "UTF8")